# Getting the relation between baroclinic streamfunction and cross-stream distance from the SOSE data

In [1]:
import pickle
import numpy as np
import xarray as xr
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import interp1d

import gsw
import csaps

import warnings

## Sample points along hydrographic section transect

In [2]:
# read in hydrographic section data
data = loadmat('data/SR1b_section/sr1b_all.mat')

icyc = np.where(data['buf_year'] != 1999)[1] # exclude 1999 # TODO: why?

buf_lat = data['buf_lat'][0,icyc]
buf_lon = data['buf_lon'][0,icyc]
buf_sal = data['buf_sal'][:,icyc]
buf_temp = data['buf_temp'][:,icyc]
buf_press = data['buf_press'].reshape(3000)
buf_year = data['buf_year'][:,icyc].reshape(316)

In [3]:
# sample points along section
start_lon_vals = []
end_lon_vals = []
start_lat_vals = []
end_lat_vals = []
for yr in np.unique(buf_year):
  idx_yr = np.where(buf_year == yr)[0]
  start_lon_vals.append(np.min(buf_lon[idx_yr]))
  end_lon_vals.append(np.max(buf_lon[idx_yr]))
  start_lat_vals.append(np.max(buf_lat[idx_yr]))
  end_lat_vals.append(np.min(buf_lat[idx_yr]))
start_lon = np.mean(np.array(start_lon_vals))
end_lon = np.mean(np.array(end_lon_vals))
start_lat = np.mean(np.array(start_lat_vals))
end_lat = np.mean(np.array(end_lat_vals))

# Extract unique points along the section
nr_points = 30
lon_along_section = np.linspace(start_lon, end_lon, nr_points)
lat_along_section = np.linspace(start_lat, end_lat, nr_points)

# calculate distance from southern end of section
dist_along_section = np.zeros(len(lat_along_section))
for i in range(len(dist_along_section)):
  dist_along_section[i] = gsw.distance(np.mean(lon_along_section)*np.ones(2), np.array([min(lat_along_section), lat_along_section[i]]))[0]

## Process the SOSE data

In [4]:
# Read in data
sose_salt_dataset = xr.open_dataset('data/SOSE_reanalysis/Salt_bsoseI155_2013to2024_monthly_DP.nc')
sose_salt = xr.DataArray(sose_salt_dataset['SALT'].values, 
                         dims=['time', 'Z', 'YC', 'XC'], 
                         coords = {'time': sose_salt_dataset['time'],
                                    'XC': sose_salt_dataset['XC'],
                                    'YC': sose_salt_dataset['YC'],
                                    'Z': sose_salt_dataset['Z']})
mask = sose_salt_dataset.maskC
sose_salt = sose_salt.where(mask)

sose_temp_dataset = xr.open_dataset('data/SOSE_reanalysis/Theta_bsoseI155_2013to2024_monthly_DP.nc')
sose_temp = xr.DataArray(sose_temp_dataset['THETA'].values, 
                         dims=['time', 'Z', 'YC', 'XC'],
                         coords = {'time': sose_temp_dataset['time'],
                                    'XC': sose_temp_dataset['XC'],
                                    'YC': sose_temp_dataset['YC'],
                                    'Z': sose_temp_dataset['Z']})
mask = sose_temp_dataset.maskC
sose_temp = sose_temp.where(mask)

sose_lat = sose_salt_dataset['YC'].values
sose_lon = sose_salt_dataset['XC'].values - 360  # convert to -180 to 180 range
sose_depth = sose_salt_dataset['Z'].values
sose_time = sose_salt_dataset['time'].values

In [5]:
# transform depth to pressure
sose_p = np.zeros((len(sose_depth), len(sose_lat)))
for i in range(len(sose_lat)):
    sose_p[:,i] = gsw.p_from_z(sose_depth, sose_lat[i])

# interpolate p to section points
p_section_sose = np.zeros((len(sose_depth), len(lat_along_section)))
for i in range(len(sose_depth)):
    p_section_sose[i,:] = np.interp(lat_along_section, sose_lat, sose_p[i,:])

In [6]:
# calculate absolute salinity and conservative temperature
# sose_SA = np.zeros(np.shape(sose_salt))
# for t in range(len(sose_time)):
#     for d in range(len(sose_depth)):
#         for i in range(len(sose_lat)):
#             for j in range(len(sose_lon)):
#                 sose_SA[t,d,i,j] = gsw.SA_from_SP(sose_salt[t,d,i,j], sose_p[d,i], sose_lon[j], sose_lat[i])
# np.save('data/SOSE_reanalysis/sose_SA.npy', sose_SA)
#sose_SA = np.load('data/SOSE_reanalysis/sose_SA.npy')  # load pre-calculated SA
sose_SA = xr.DataArray(np.load('data/SOSE_reanalysis/sose_SA.npy'), 
                        dims=['time', 'depth', 'latitude', 'longitude'],
                        coords={'time': sose_time,
                                'longitude': sose_lon,
                                'latitude': sose_lat,
                                'depth': sose_depth})
sose_CT = gsw.CT_from_pt(sose_SA, sose_temp.values)

In [7]:
# interpolate SA and CT to section points
SA_section_sose = np.zeros((len(sose_time), len(sose_depth), len(lat_along_section)))
CT_section_sose = np.zeros((len(sose_time), len(sose_depth), len(lat_along_section)))
points = np.array([(lat, lon) for lat,lon in zip(lat_along_section,lon_along_section)])
for i in range(len(sose_time)):
    for j in range(len(sose_depth)):
        f_SA_sose = RegularGridInterpolator((sose_lat, sose_lon), sose_SA.values[i,j,:,:])
        f_CT_sose = RegularGridInterpolator((sose_lat, sose_lon), sose_CT.values[i,j,:,:])
        SA_section_sose[i,j,:] = f_SA_sose(points)
        CT_section_sose[i,j,:] = f_CT_sose(points)

# interpolate in p (to high resolution)
SA_section_interp_sose = np.zeros((len(sose_time), len(buf_press), len(lat_along_section)))
CT_section_interp_sose = np.zeros((len(sose_time), len(buf_press), len(lat_along_section)))
for i in range(len(sose_time)):
    for j in range(len(lat_along_section)):
        f_SA_sose = interp1d(p_section_sose[:,j], SA_section_sose[i,:,j], bounds_error=False, fill_value=np.nan)
        f_CT_sose = interp1d(p_section_sose[:,j], CT_section_sose[i,:,j], bounds_error=False, fill_value=np.nan)
        SA_section_interp_sose[i,:,j] = f_SA_sose(buf_press)
        CT_section_interp_sose[i,:,j] = f_CT_sose(buf_press)

In [8]:
# compute baroclinic streamfunction
gpan_sose = np.zeros((len(sose_time), len(buf_press), len(lat_along_section)))
for i in range(len(sose_time)):
    gpan_sose[i,:,:] = -gsw.geo_strf_dyn_height(SA_section_interp_sose[i,:,:], CT_section_interp_sose[i,:,:], buf_press, 1500)

sose_phi1500 = abs(gpan_sose[:,249,:])

# make spline fit through relationship between phi1500 and distance
dist_repeat = np.tile(dist_along_section, len(sose_time))
sose_phi1500_flatten = sose_phi1500.flatten()
pp_dist_sose = csaps.csaps(sose_phi1500_flatten[np.argsort(sose_phi1500_flatten)[:-np.count_nonzero(np.isnan(sose_phi1500_flatten))]],
                           dist_repeat[np.argsort(sose_phi1500_flatten)[:-np.count_nonzero(np.isnan(sose_phi1500_flatten))]],
                           smooth = 0.85)

# save the spline fit
with open('data/SOSE_reanalysis/pp_dist_sose.pkl', 'wb') as f:
    pickle.dump(pp_dist_sose, f)

/nethome/5664187/miniconda3/envs/carlo/lib/python3.13/site-packages/csaps/_sspumv.py:301: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = la.spsolve(a, b)
